## In diesem Skript werden die bereinigten Batteriedaten der Battery1 analyisiert

In [ ]:
### Importieren der verwendeten Bibilotheken
import os
import glob

import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

In [ ]:
plt.style.use(['mystyle'])

### Einlesen der benötigten Daten

In [ ]:
pathToPickle_battery_state = os.path.join("..", "..", "data", "TUDA_battery_state", "all_TUDA_battery_states_battery_state_clean.pickle")

# Einlesen der Daten
df_battery_state = pd.read_pickle(pathToPickle_battery_state)

# Den "wagon_IDs" einen "wagon_type" mittels Mapping zuordnen
pathToMappingPickle = os.path.join("..", "..", "data", "mappingDf.pickle")
mappingdf = pd.read_pickle(pathToMappingPickle)
df_battery_state = pd.merge(df_battery_state, mappingdf, on = "wagon_ID")

# provider ist doppelt und wird gedroppt
df_battery_state.drop("provider_y", axis=1, inplace=True)
df_battery_state.rename(columns={'provider_x': 'provider'}, inplace=True)

### Daten auf die verschiednene Wagon Typen aufteilen

In [ ]:
# Liste mit allen wagon_types wird erstellt
wagon_type_list = np.sort(df_battery_state["wagon_type"].drop_duplicates().to_numpy())
wagon_type_list

In [ ]:
# Liste mit einem DataFrame für jeden 'wagon_type' wird erstellt
df_battery_state_list = []
for wagon_type in wagon_type_list:
    df_battery_state_wagon_type = df_battery_state.loc[df_battery_state['wagon_type'] == wagon_type]
    df_battery_state_list.append(df_battery_state_wagon_type)
    

### Vorlage für die statistischen Analysen

In [ ]:
def CalculateBatteryStateStatistics(dataFrame, BatteryTypeStr = 'battery_state'):
    '''
    Funktion, die sowohl für Battery als auch Battery2 Statistiken
    für einen bestinnen Zeitraum erstellen kann
    input:
        dataFrame
        'battery_state' / 'battery2_state' --> Auswahl des Batteriezustands; Standard: 'battery_state'
    '''
    #bins erstellen, in die die Daten eingeteilt und anschließend gemittelt werden sollen
    binSize = 1 # hier kann die Bingröße verändert werden; sinnvolle Werte sind binSize = 1 für einen Tag und binSize = 7 für eine Woche
    binsSeconds = np.arange(0, 50*7*24*60*60/binSize, binSize*24*60*60)
    binsSecondsLabels = [i for i, bin in enumerate(binsSeconds)]
    binsSeconds = binsSeconds.tolist()
    binsSecondsLabels.pop() # ein Wert muss entfernt werden, um gleich viele Labels wie bins zu definieren!
    # Daten mittels der timestamps in bins einteilen (jeweils immer genau einen bin zuweisen)
    dataFrame['Tag'] = pd.cut(dataFrame['timestamp_measure_battery'], bins = binsSeconds, labels = binsSecondsLabels)
    
    batteryStateMeans = []
    for binNumber in binsSecondsLabels:
        # prüft für jede Reihe ob der bin der entsprechenden binnumber die gerade ausgewählt ist entspricht
        dfFiltered = dataFrame.loc[dataFrame['Tag'] == binNumber] 
     
        batteryStateMeans.append([binNumber, dfFiltered[BatteryTypeStr].mean(), dfFiltered[BatteryTypeStr].std(), len(dfFiltered)])
    #DataFrame aus den Werten zusammensetzen
    batteryStateMeansDf = pd.DataFrame(batteryStateMeans, columns = ['start_day_bin', BatteryTypeStr + '_mean',BatteryTypeStr + '_std', 'binlength'])
    
    return batteryStateMeansDf

### Statistische Analyse des gesamten DataFrames von battery (ohne Einteilung / Zuweisung in verschiedene Kategorien)

In [ ]:
battery_state_mean_df = CalculateBatteryStateStatistics(df_battery_state)

In [ ]:
# Bins herausnehmen, an denen zu wenige Daten vorliegen
battery_state_mean_df.drop(battery_state_mean_df.loc[battery_state_mean_df['binlength']<=10000].index, inplace=True)


In [ ]:
fig, ax = plt.subplots(ncols = 2, figsize=(30,7))
ax[0].plot(battery_state_mean_df['start_day_bin'], battery_state_mean_df['battery_state_mean'], marker='o')
ax[1].scatter(battery_state_mean_df['start_day_bin'], battery_state_mean_df['battery_state_mean'], c = battery_state_mean_df['binlength'], marker='o', cmap = 'jet')


### Statistische Analyse für die verschiedenen 'wagon_types'

In [ ]:
batteryStatisticsWagonTypesList = []

for dfBatteryState in df_battery_state_list:
    batteryStatisticsWagonTypesList.append(CalculateBatteryStateStatistics(dfBatteryState))
    

In [ ]:
fig, ax = plt.subplots(ncols = 4, nrows=2, figsize=(20,8), sharex=True, sharey=True)
ax = ax.flatten()

i = 0
for i, batteryStatisticWagonType in enumerate(batteryStatisticsWagonTypesList):
    # Tage, an denen zu wenige Messwerte vorliegen vernachlässigen
    batteryStatisticWagonType.drop(batteryStatisticWagonType.loc[batteryStatisticWagonType['binlength']<=1000].index, inplace=True)

    ax[i].plot(batteryStatisticWagonType['start_day_bin'], batteryStatisticWagonType['battery_state_mean'], marker='o')
    ax[i].plot(batteryStatisticWagonType['start_day_bin'], batteryStatisticWagonType['battery_state_mean']+batteryStatisticWagonType['battery_state_std'], color='grey')
    ax[i].plot(batteryStatisticWagonType['start_day_bin'], batteryStatisticWagonType['battery_state_mean']-batteryStatisticWagonType['battery_state_std'], color='grey')
    
    ax[i].fill_between(batteryStatisticWagonType['start_day_bin'], batteryStatisticWagonType['battery_state_mean']+batteryStatisticWagonType['battery_state_std'],
                       batteryStatisticWagonType['battery_state_mean']-batteryStatisticWagonType['battery_state_std'], color='lightgrey', alpha=0.3)

    
    ax[i].set_title(f'Wagontyp {i+1}')
    if i >= 4:
        ax[i].set_xlabel('Tag')
      
ax[0].set_ylabel(r'Batterieladezustand ($\%$)')
ax[4].set_ylabel(r'Batterieladezustand ($\%$)')

### Statistische Analyse für die verschiedenen Provider

In [ ]:
# Liste mit allen wagon_types wird erstellt
provider_list = np.sort(df_battery_state["provider"].drop_duplicates().to_numpy())
provider_list = provider_list[provider_list>0]
provider_list

In [ ]:
# Liste mit einem DataFrame für jeden 'wagon_type' wird erstellt
df_provider_list = []
for provider in provider_list:
    df_provider = df_battery_state.loc[df_battery_state['provider'] == provider]
    df_provider_list.append(df_provider)
df_provider_list[0]

In [ ]:
batteryStatisticsProviderList = []

for dfProvider in df_provider_list:
    batteryStatisticsProviderList.append(CalculateBatteryStateStatistics(dfProvider))
batteryStatisticsProviderList

In [ ]:
fig, ax = plt.subplots(ncols = 3, nrows=1, figsize=(7,3), sharex=True, sharey=True)
ax = ax.flatten()

i = 0
for i, batteryStatisticProvider in enumerate(batteryStatisticsProviderList):
    # Tage, an denen zu wenige Messwerte vorliegen vernachlässigen
    batteryStatisticProvider.drop(batteryStatisticProvider.loc[batteryStatisticProvider['binlength']<=1000].index, inplace=True)

    ax[i].plot(batteryStatisticProvider['start_day_bin'], batteryStatisticProvider['battery_state_mean'], marker='o')
    ax[i].plot(batteryStatisticProvider['start_day_bin'], batteryStatisticProvider['battery_state_mean']+batteryStatisticProvider['battery_state_std'], color='grey')
    ax[i].plot(batteryStatisticProvider['start_day_bin'], batteryStatisticProvider['battery_state_mean']-batteryStatisticProvider['battery_state_std'], color='grey')
    
    ax[i].fill_between(batteryStatisticProvider['start_day_bin'], batteryStatisticProvider['battery_state_mean']+batteryStatisticProvider['battery_state_std'],
                       batteryStatisticProvider['battery_state_mean']-batteryStatisticProvider['battery_state_std'], color='lightgrey', alpha=0.3)

    
    ax[i].set_title(f'Provider {provider_list[i]}')
    #if i >= 4:
    ax[i].set_xlabel('Tag')
ax[0].set_xticks(range(0,350,50))
ax[0].set_ylabel(r'Batterieladezustand in \%')
# Plots speichern, falls erwünscht, auf richtigen Pfad achten 
#fig.savefig(os.path.join("..", "..", "data", "battie1_analyse_year.pdf"))
#fig.savefig(os.path.join("..", "..", "data", "battie1_analyse_year.png"))

___

### Analyse der einzelnen Ausfälle von Messboxen

+ Alle Einträge raussuchen, bei den die Battery voll entladen ist

In [ ]:
df_battery_state['Ausfall_Batterie'] = 0
df_battery_state['Ausfall_Batterie'].loc[df_battery_state.battery_state==0] = 1

In [ ]:
len_0 = len(df_battery_state.loc[df_battery_state.Ausfall_Batterie == 1])
len_n0 = len(df_battery_state.loc[df_battery_state.Ausfall_Batterie == 0])

print('Es sind '+str(len_0/(len_0+len_n0)*100)+' % der Battery_state Einträge = 0')

+ Alle Einträge raussuchen, bei den die Battery-Differenz negativ ist

In [ ]:
df_battery_state['Diff_Bat_State'] = 0
df_battery_state.sort_values(['wagon_ID','timestamp_measure_battery'])
diff_bat_state = df_battery_state.battery_state.diff()
df_battery_state['Diff_Bat_State'].loc[diff_bat_state<0] = 1

+ Analyse, ob der leere Batteriezustand ein Datenfehler ist, oder durch zulange Entladung erfolgt ist

     + State1: Batterie leer und Batterie_Differenz nicht 0 (da sonst mehrere Zeitpunkte mit 0 hintereinander einzeln gezählt würden
     + State2: Batterie Differenz von den letzten 3 Einträgen muss auch negativ sein, damit ein kontinuierlicher Abfall gewährleistet werden kann

In [ ]:
# neues feature Batterie_Entladen erzeugt
df_battery_state['Batterie_Entladen'] = 0
# Indizes, die auf State1 zutreffen
ind_state1   = np.where((df_battery_state.Ausfall_Batterie == 1) & (df_battery_state.Diff_Bat_State == 1))
# Indizes, die als Bedingung für State2 zählen
ind_stateI   = np.add(ind_state1,-1)[0]
ind_stateII  = np.add(ind_state1,-2)[0]
ind_stateIII = np.add(ind_state1,-3)[0]
# boolean Listen, die Anzeigen, ob State 2 erfüllt ist
state1 = (df_battery_state.iloc[ind_stateI]['Diff_Bat_State'] == 1).reset_index(drop=True)
state2 = (df_battery_state.iloc[ind_stateII]['Diff_Bat_State'] == 1).reset_index(drop=True)
state3 = (df_battery_state.iloc[ind_stateIII]['Diff_Bat_State'] == 1).reset_index(drop=True)
# Indizes, die State2 erfüllt haben
ind_state2 = ind_state1[0][(state1 & state2 & state3)]
# feature = 1, wo sich die Batterie entleert hat
df_battery_state['Batterie_Entladen'].loc[df_battery_state.iloc[ind_state2].index] = 1

In [ ]:
len_0 = len(df_battery_state.loc[df_battery_state.Batterie_Entladen == 1])

print('Es haben sich '+str(len_0)+' mal Batterien in dem untersuchten Zeitraum entladen')

In [ ]:
df_battery_state.loc[df_battery_state.Batterie_Entladen == 1]

#### Plotten des Entladevorgangs der Batterie-Entladungen

In [ ]:
df_BatAusfall = df_battery_state.loc[df_battery_state.Batterie_Entladen == 1]
list_BatAusfall = []

for ii, index in enumerate(df_BatAusfall.index):
    indexList = list(range(index-10,index+2))
    list_BatAusfall.append(df_battery_state.loc[indexList])

In [ ]:
for df in list_BatAusfall:
    if len(df.wagon_ID.unique()) > 1:
        print(df)

In [ ]:
list_BatAusfall[0]

In [ ]:
fig, ax = plt.subplots(figsize=[3.5,3])
for df in list_BatAusfall[:4]:
    ax.plot(np.add(df['Woche'].tolist(),-df['Woche'].iloc[10].tolist()), df.battery_state)
ax.set_xlabel('Zeit relativ zum Batterieausfall in Wochen')
ax.set_ylabel('Batterieladezustand in \%')
ax.grid(True)

___

## Analyse Einfluss Battery2 auf Battery

### Entferne alle Einträge von df in denen battery2 gleich nan oder kleiner 0

In [ ]:
df_allBat.loc[df_battery_state['battery2_state'].isna()].index

In [ ]:
df_bat2Ana = df_battery_state

df_bat2Ana.drop(df_bat2Ana.loc[df_bat2Ana['battery_state']<0].index, inplace=True)
df_bat2Ana.drop(df_bat2Ana.loc[df_bat2Ana['battery2_state'].isna()].index, inplace=True)
# kleiner GLEICH 0 gedropped, da andere Provider =0 auch für Batterie 2 angegeben haben
df_bat2Ana.drop(df_bat2Ana.loc[df_bat2Ana['battery2_state']<=0].index, inplace=True)

### Ermittlung des Providers von battery2

In [ ]:
prov_bat2 = pd.unique(df_bat2Ana['provider'])
print('Der Provider von der 2. Batterie ist Provider', str(prov_bat2))
df_bat2Ana.drop(df_bat2Ana.loc[df_bat2Ana['provider']==0].index, inplace=True)